In [1]:
import lyricsgenius as lg 
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd 

# Genuis Lyric Extraction

In [3]:
# set up api keys to use genuis api 
api_key = "bQamoWDa9VfHDPGGamXTbpHdhE1-ld4m1Z9pBA4tfYHzXqbyBe0Rjn8Tk6Rag5zy"
genius = lg.Genius(api_key)
genius = lg.Genius(api_key, skip_non_songs=True, excluded_terms=["(Remix)", "(Live)"], remove_section_headers=True)


In [9]:
# set up Spotify API credentials for later
spotify_client_id  = "ea6cf3328b714f6dbf61d38ccc7d6715"
spotify_client_secret = "0651f5daaaa843cb9975b19daaef62f9"
TOP_SONGS_URI = "5ABHKGoOzxkaa28ttQV9sE" # This is the uri for the top songs playlist (link: https://open.spotify.com/playlist/5ABHKGoOzxkaa28ttQV9sE?si=c3b9928b999c4462)

client_credentials_manager = SpotifyClientCredentials(client_id=spotify_client_id, client_secret=spotify_client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [10]:
# define function to get lyrics from genius api 

def get_lyrics(artist_name, artist_song):
    # Search for artist and song
    try:
        song = genius.search_song(artist_song, artist_name)

        if song:
            return song.lyrics
    except:
        return 


# Spotify Genre Extraction + Dataset Generator

In [11]:
# get the genre of artist 
def get_artist_genres(artist_name):

    # Search for the artist
    results = sp.search(q=f"artist:{artist_name}", type='artist')

    # Check if there are artist results
    if 'artists' in results and 'items' in results['artists'] and results['artists']['items']:
        # Get the first artist (assuming it's the correct one)
        artist_id = results['artists']['items'][0]['id']

        # Get detailed information about the artist
        artist_info = sp.artist(artist_id)

        # Get the genres associated with the artist
        genres = artist_info.get('genres', 'Genres not found.')
        return genres
    else:
        return None

In [12]:
def make_data_set():
 
    # Get the top 100 tracks globally
    results = sp.playlist_tracks(TOP_SONGS_URI)  

    # Extract track names and artists
    tracks = results['items']
    
    # 
    df = pd.DataFrame(columns=['song', 'artist', 'genre', 'lyrics'])

    for track in tracks:
        
        # get song info for track 
        song = track['track']['name']     
        artist = track['track']['artists'][0]['name']
        genre = get_artist_genres(artist)
        lyrics = get_lyrics(artist, song)
        
        # store in dataframe
        if genre and lyrics:
            print("Successfully added info for:", song)
            df.loc[len(df.index)] = [song, artist, genre, lyrics]
        else:
            print("Error adding:", song)
    
    df.to_csv("custom_dataset.csv", index=False)
        
    return df

make_data_set()

Searching for "Blinding Lights" by The Weeknd...
Done.
Successfully added info for: Blinding Lights
Searching for "Shape of You" by Ed Sheeran...
Error adding: Shape of You
Searching for "Someone You Loved" by Lewis Capaldi...
Done.
Successfully added info for: Someone You Loved
Searching for "Sunflower - Spider-Man: Into the Spider-Verse" by Post Malone...
Done.
Successfully added info for: Sunflower - Spider-Man: Into the Spider-Verse
Searching for "Dance Monkey" by Tones And I...
Done.
Successfully added info for: Dance Monkey
Searching for "One Dance" by Drake...
Done.
Successfully added info for: One Dance
Searching for "STAY (with Justin Bieber)" by The Kid LAROI...
Done.
Successfully added info for: STAY (with Justin Bieber)
Searching for "As It Was" by Harry Styles...
Done.
Successfully added info for: As It Was
Searching for "Starboy" by The Weeknd...
Done.
Successfully added info for: Starboy
Searching for "rockstar" by Post Malone...
Done.
Successfully added info for: rockst